In [ ]:
import numpy as np
import pandas as pd
import time
import random
import os

# Mouse mPFC data by STARmap
## single  method

In [ ]:
os.chdir('STCC\\result\\single method\\')
methods = ['BASS','BayesSpace','SpatialPCA','SEDR','SpaGCN','STAGATE']
rep_list = ['20180417_BZ5_control','20180419_BZ9_control','20180424_BZ14_control']
k_list = np.arange(10,21,1)
consensus_list = ['Average-based','Onehot-based','wNMF-based']

for method in methods:
    print(method)
    for rep in rep_list:
        print(rep)
        gt = pd.read_csv(f'../../data/gt_c_{rep}.csv',index_col=0)
        ground_truth = [i for i in gt['x']]
        
        df_result = pd.DataFrame()
        with open(f"consensus_{method}_{rep}_ari.txt", "w") as f:
            f.write("method\tn_cluster\tseed\tari_value\n")
            for k in k_list:
                if method == 'SEDR':
                    df = pd.read_csv(f'../../result/{method}/{rep}_k={k}.csv',sep='\t')
                else:
                    df = pd.read_csv(f'../../result/{method}/{rep}_k={k}.csv',index_col=0)
                for consensus in consensus_list:
                    for seed in range(10): 
                        if consensus == 'wNMF-based':
                            labels_consensus, contributions = consensus_STCC(df,k,methods=consensus,seed=seed)
                            print(f'weight of {method} is:{contributions}')
                        else:
                            labels_consensus = consensus_STCC(df,k,methods=consensus,seed=seed)
                        df_result['k='+str(k)+ '_'+consensus+ '_' +str(seed)] = labels_consensus

                        ari_value = adjusted_rand_score(ground_truth,labels_consensus)
                        f.write(f"{consensus}\t{str(k)}\t{str(seed)}\t{str(ari_value)}\n")
                        print(f'consensus = {consensus},n_cluster = {k},seed = {seed},ari = {ari_value}')
        df_result.to_csv(f'consensus_{method}_{rep}_labels.csv')

## all methods

In [ ]:
# merge data
os.chdir('STCC\\result\\all methods\\')
methods = ['BASS','BayesSpace','SpatialPCA','SEDR','SpaGCN','STAGATE']
rep_list = ['20180417_BZ5_control','20180419_BZ9_control','20180424_BZ14_control']
k_list = np.arange(10,21,1)

for rep in rep_list:
    print(rep)
    for k in k_list:
        dff = pd.DataFrame()
        for method in methods:
            if method == 'SEDR':
                df = pd.read_csv(f'../../result/{method}/{rep}_k={k}.csv',sep='\t')
            else:
                df = pd.read_csv(f'../../result/{method}/{rep}_k={k}.csv',index_col=0)
            collen = df.shape[1]
            for i in range(collen):
                dff[method+'_'+str(i)] = df.iloc[:,i].tolist()
        dff.to_csv(f'all_methods_{rep}_k={k}.csv')

In [ ]:
os.chdir('STCC\\result\\all methods\\')
rep_list = ['20180417_BZ5_control','20180419_BZ9_control','20180424_BZ14_control']
k_list = np.arange(10,21,1)
consensus_list = ['Average-based','Onehot-based','wNMF-based']

for rep in rep_list:
    print(rep)
    gt = pd.read_csv(f'../../data/gt_c_{rep}.csv',index_col=0)
    ground_truth = [i for i in gt['x']]

    df_result = pd.DataFrame()
    with open(f"consensus_{rep}_ari.txt", "w") as f:
        f.write("method\tn_cluster\tseed\tari_value\n")
        for k in k_list:
            df = pd.read_csv(f'all_methods_{rep}_k={k}.csv',index_col=0)
            for consensus in consensus_list:
                for seed in range(10): 
                    if consensus == 'wNMF-based':
                        labels_consensus, contributions = consensus_STCC(df,k,methods=consensus,seed=seed)
                        print(f'weight of {consensus} is:{contributions}')
                    else:
                        labels_consensus = consensus_STCC(df,k,methods=consensus,seed=seed)
                    df_result['k='+str(k)+ '_'+consensus+ '_' +str(seed)] = labels_consensus

                    ari_value = adjusted_rand_score(ground_truth,labels_consensus)
                    f.write(f"{consensus}\t{str(k)}\t{str(seed)}\t{str(ari_value)}\n")
                    print(f'consensus = {consensus},n_cluster = {k},seed = {seed},ari = {ari_value}')
    df_result.to_csv(f'consensus_{rep}_labels.csv')